# Packages
1. __OpenCV__ - *Open Source Computer Vision Library, is an open source computer vision and machine learning software library ... The library has more than 2500 optimized algorithms ... These algorithms can be used to detect and recognize faces, identify objects, classify human actions in videos, track camera movements, track moving objects, etc.* - opencv.org
2. __Numpy__ - *NumPy, is the fundamental package for scientific computing with Python ... NumPy can also be used as an efficient multi-dimensional container of generic data.* - numpy.org
3. __OS__ - *OS, is a built-in module from Python. The functions OS provides, allows for interfacing with the underlying operating system Python is running on (e.g. Mac, Windows, Linux).*
4. __PIL & Image__ - *Python Imaging Library (PIL) adds support for opening, manipulating, and saving many different image file formats. Image is a Module within the Library.*
5. __Pickle__ - *Pickle converts Python objects (object from a class, variable, list, etc.) to byte stream and then back again. With Machine Learning, saving Classifier.*

In [ ]:
# Import packages.
import cv2
import numpy as np
import os
from PIL import Image
import pickle

BASE_DIR = os.getcwd()    # get the path location of this python file (or the path location to this notebook).
image_dir = os.path.join(BASE_DIR, "images")    # get path location of 'images' file.

# Local Binary Patterns Histograms (LBPH) Face Recognizer

![title](Notebook_Materials/LBP_Binary.png)
__LBP conversion to binary__. *Source: López & Ruiz; Local Binary Patterns applied to Face Detection and Recognition.* 

![title](Notebook_Materials/Sample_Histogram.png)
__Sample Histogram__. *Source: superdatascience.com/opencv-face-recognition*

![title](Notebook_Materials/Principal_Components.png)
__LBPH Face Recognizer Principal Components__. *Source: docs.opencv.org*


In [ ]:
# OpenCV has pre-trained classifier for detecting objects. In this case, faces (frontal).
face_cascade = cv2.CascadeClassifier("cascades/data/haarcascade_frontalface_alt2.xml")    # load the XML face 
# (frontal) classifier.

recognizer = cv2.face.LBPHFaceRecognizer_create()    # creating an LBPH (Local Binary Patterns Histograms) Face
# Recognizer from OpenCV.

x_train = []    # initializing an empty list for the NUMPY arrays (i.e. pixel values) that correspond to the specific
# faces within the images.
y_labels = []    # initializing an empty list for image label ID's.
current_id = 0    # initializing label ID's to 0.
labels_ids = {}    # initializing empty dictionary for label-ID (KEY=label, VALUE=ID).

# Prepare Data

## Looping
    Loop through the folders in 'images'.
        Loop through each folder found in 'images'.
            If a file (image) ends with *png*, *jpg*, or *jpeg*, then execute the body of code.

### Each iteration
    1. Get path location of image/ label of image.
    2. Assign an ID to the image.
    3. Append to a dictionary the ID that corresponds to a specific Label (name).
    4. Convert image to grayscale & resize to (300, 300).
    5. Turn image into NUMPY array.
    6. Face Detection Loop.
        A. Detect ROI (face) within the image.
        B. Append the NUMPY array of the ROI to 'x_train'.
        C. Append the corresponding ID of the image to 'y_labels'.

In [ ]:
# Go through EACH image folder within 'images'. Get path of each image & label (name of person in image as well as
# name of the folder that image is located in) + MORE.
for root, dirs, files in os.walk(image_dir):
    # Go through EACH image.
    for file in files:
        if file.endswith("png") or file.endswith("jpg") or file.endswith("jpeg"):
            img_path = os.path.join(root, file)    # get path location of image.
            img_label = os.path.basename(root).upper()    # get label of image i.e. name of person the image is showing.

            # If the label of the image is not already in the dictionary (which all won't be to begin with), then assign
            # the value of the 'current_id' to that label in the dictionary. Then, current_id++.
            if img_label not in labels_ids:
                labels_ids[img_label] = current_id
                current_id += 1
            id_ = labels_ids[img_label]    # 'id_' is the value associating to a certain label in the dictionary.
            
            print(id_, img_label, img_path)

            pil_image = Image.open(img_path).convert("L")    # returns the image corresponding to the path in grayscale.
            size = (300, 300)    # resize all images to this dimension.
            final_img = pil_image.resize(size, Image.ANTIALIAS)
            img_array = np.array(final_img, "uint8")    # turns the image into NUMPY array.

            # Face detection from image array.
            faces = face_cascade.detectMultiScale(img_array, scaleFactor=1.3, minNeighbors=5)
            for (x, y, w, h) in faces:
                roi = img_array[y:y+h, x:x+w]
                x_train.append(roi)    # add to the 'x_train' list the ROI (face), but in a NUMPY array.
                y_labels.append(id_)    # add to the 'y_labels' list the ID's associating to each label.

# Save Labels & ID's

In [ ]:
# Using Pickle to save the label-ID dictionary in order to use in the Model.
with open("labels_ids.pickle", "wb") as f:
    pickle.dump(labels_ids, f)

# Train & Save Data

In [ ]:
# Training the recognizer with the training data (NUMPY arrays of the ROI's) and the ID's (in a NUMPY array).
recognizer.train(x_train, np.array(y_labels))
recognizer.save("training_data.yml")    # saving the trained data file as .yml.